
# Getting started

Once you've chosen your scenario, download the data from the Iowa website in csv format. Start by loading the data with pandas. You may need to parse the date columns appropriately.

In [4]:
import pandas as pd

## Load the data into a DataFrame
# pd.read_csv()

## Transform the dates if needed, e.g.
# df["Date"] = pd.to_datetime(df["Date"], format="%m-%d-%y")
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression



In [ ]:
dataset=pd.read_csv('iowa_liquor_sales_proj_2')

# Explore the data

Perform some exploratory statistical analysis and make some plots, such as histograms of transaction totals, bottles sold, etc.

In [1]:

dataset_with_target=dataset.copy()
dataset_with_target.dropna(inplace=True)


dataset_with_target['Zip Code'] = dataset_with_target['Zip Code'].replace('712-2',7122)  
dataset_with_target['Zip Code'] = dataset_with_target['Zip Code'].astype('int32')
dataset_with_target['Sale (Dollars)'] = dataset_with_target['Sale (Dollars)'].replace({'\$':''}, regex=True)
dataset_with_target['Sale (Dollars)'] = dataset_with_target['Sale (Dollars)'].astype('float32')
dataset_with_target['State Bottle Cost'] = dataset_with_target['State Bottle Cost'].replace({'\$':''}, regex=True)
dataset_with_target['State Bottle Cost'] = dataset_with_target['State Bottle Cost'].astype('float32')
dataset_with_target['State Bottle Retail'] = dataset_with_target['State Bottle Retail'].replace({'\$':''}, regex=True)
dataset_with_target['State Bottle Retail'] = dataset_with_target['State Bottle Retail'].astype('float32')
dataset_with_target['City']=dataset_with_target['City'].str.lower()
dataset_with_target['Category Name']=dataset_with_target[ 'Category Name'].str.lower()
dataset_with_target['County']=dataset_with_target['County'].str.lower()
dataset_with_target['Item Description']=dataset_with_target['Item Description'].str.lower()

In [ ]:
dataset_with_target['Profit in %']=(dataset_with_target['State Bottle Retail']-dataset_with_target['State Bottle Cost'])*(dataset_with_target['State Bottle Retail']**(-1))*100
dataset_with_target['Volume Sold (Mililiters)']=dataset_with_target['Volume Sold (Liters)']*1000
dataset_with_target['Volume Sold (Mililiters)'] = dataset_with_target['Volume Sold (Mililiters)'].apply(lambda x: (x-100)/2507900)
dataset_with_target['Sale (Dollars)'] = dataset_with_target['Sale (Dollars)'].apply(lambda x: (x-1.34)/36390.66)


In [ ]:
print(dataset_with_target['Sale (Dollars)'].describe())
print(dataset_with_target['State Bottle Cost'].describe())
print(dataset_with_target['State Bottle Retail'].describe())
print(dataset_with_target['Profit in %'].describe())


# Record your findings

Be sure to write out anything observations from your exploratory analysis.


The were little null values 
some data types needed to be converted 

# Mine the data

Now you are ready to compute the variables you will use for your regression from the data. For example, you may want to compute total sales per store from Jan to March of 2015, mean price per bottle, etc. Refer to the readme for more ideas appropriate to your scenario.

Pandas is your friend for this task. Take a look at the operations here for ideas on how to make the best use of pandas and feel free to search for blog and Stack Overflow posts to help you group data by certain variables and compute sums, means, etc. You may find it useful to create a new data frame to house this summary data.
 


In [ ]:
df1=dataset_with_target.groupby('City')[('Sale (Dollars)')].sum().reset_index()
df1['Sale (Dollars)']=df1['Sale (Dollars)'].apply(lambda x: (x*36390.66)+1.34)
df2=dataset_with_target.groupby('Category Name')[('Sale (Dollars)')].sum().reset_index()
df2['Sale (Dollars)']= df2['Sale (Dollars)'].apply(lambda x:(x*36390.66)+1.34)



s1= df1[df1['Sale (Dollars)'] > 500000].sort_values(by='Sale (Dollars)',ascending=True)
y_pos1 = np.arange(len(s1)) 
list1=['Des Moines', 'Cedar Rapids','Davenport','Iowa City','Waterloo','Sioux City','Council Bluffs','West Des Moines','Dubuque','Ames','Ankeny','Bettendorf','Cedar Falls','Coralville','Windsor Heights','Mason City','Fort Dodge']
plt.barh(y_pos1,s1['Sale (Dollars)'], align='center', alpha=0.5)
list1.reverse()
plt.yticks(y_pos1, list1)
plt.xlabel('Sales')
plt.title('Sales per City')
plt.show()

s2= df2[df2['Sale (Dollars)'] > 500000].sort_values(by='Sale (Dollars)',ascending=True)
y_pos2 = np.arange(len(s2)) 
plt.barh(y_pos2,s2['Sale (Dollars)'], align='center', alpha=0.5)
plt.yticks(y_pos2, df2['Category Name'])
plt.xlabel('Sales')
plt.title('Sales per Liquor')
plt.show()

# Refine the data

Look for any statistical relationships, correlations, or other relevant properties of the dataset.

There were little correlation amongst variable except the odbious ones like volume sold and sales


# Build your models

Using scikit-learn or statsmodels, build the necessary models for your scenario. Evaluate model fit.

In [ ]:
y0=dataset_with_target['Sale (Dollars)'].values

X1=dataset_with_target['Store Number','County,','Category Name','Vendor Number','Item Description'
                       ,'Bottle Volume (ml)','State Bottle Retail', 'Bottles Sold'].values


labelencoder_X=LabelEncoder()
X1[:,1]=labelencoder_X.fit_transform(X1[:,1])
X1[:,2]=labelencoder_X.fit_transform(X1[:,2])
X1[:,4]=labelencoder_X.fit_transform(X1[:,4])



X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y0, test_size = 0.2, random_state = 0)
regressor1=LinearRegression()
regressor1.fit(X_train1,y_train1)


#Predicting the test set results
y_pred1=regressor1.predict(X_test1)



Use statmodel to verify the model perfomance and features p values

In [ ]:
X_opt2=np.append(arr=np.ones((269258,1)).astype(int),values=X1,axis=1).astype(float)
regressor_OLS1=sm.OLS(y0,X_opt2).fit()
regressor_OLS1.summary()


In [ ]:
reg1 = reg = linear_model.Ridge(alpha=0.00000005, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=True, random_state=0, solver='auto', tol=0.000000000001)
        
reg1.fit(X_train1,y_train1)
y_pred2=reg1.predict(X_test1)

metrics.r2_score(y_pred = y_pred2,
                 y_true = y_test1)


Tried some regulation but the R square was lower than with simple linear regression

# Plot your results

Again make sure that you record any valuable information. For example, in the tax scenario, did you find the sales from the first three months of the year to be a good predictor of the total sales for the year? Plot the predictions versus the true values and discuss the successes and limitations of your models

In [ ]:
dataset2=pd.DataFrame()
dataset2['Predictions']=y_pred1
dataset2['Sales']=y_test1
res=y_test1-y_pred1
dataset2['Residual']=res
dataset2['Residual abs']=abs(res)
sns.set(color_codes=True)
g = sns.lmplot(x="Residual", y="Sales", data=dataset2)
plt.show()

-R square is a limitation
-The absolut errors is very good

# Present the Results

Present your conclusions and results. If you have more than one interesting model feel free to include more than one along with a discussion. Use your work in this notebook to prepare your write-up.

-City and Category were importan factors
-The profit margin is very constant, try some feature engineering but didn't  found very useful.

-Best cities Dubuque, Ames and Ankeny
-Best volume 750 ml and up
-Around to 10.00 since imported drinks sell very well